In [ ]:
import numpy as np
import scipy.signal as sig
import bokeh.plotting as bkp
import bokeh.models as bkm
bkp.output_notebook()

# import os
# os.environ['ARRAY_MODULE'] = 'numpy'
from asl_bloch_sim import bloch, rf
from asl_bloch_sim import xp, asnumpy

In [ ]:
duration = 0.08 # seconds
dt = 0.00001 # seconds
B0 = 3 # Tesla

flip_angle = 180 # degrees
rf_duration = 0.012 # seconds
rf_bandwidth = 2000 # Hz
rf_stretch = 2.5
rf_amplitude = 3e-5 # T

off_resonance = 1600 # Hz
spectrum_lines = 600
B1_inhomogeneity = np.linspace(0.01, 1, 100) # fraction of B1

T1 = 1.5 # seconds
T2 = 0.1 # seconds

In [ ]:
time = np.arange(0, duration, dt) # seconds
rf_time = np.arange(-rf_duration / 2, rf_duration / 2, dt)

rf_pulse_am, rf_pulse_fm = rf.adiabatic_pulse(flip_angle, rf_duration,
                                              rf_bandwidth, rf_stretch, dt, amplitude=rf_amplitude)
rf_am = rf.extend(rf_pulse_am, duration, dt)
rf_fm = rf.extend(rf_pulse_fm, duration, dt)

dfz = np.linspace(0, off_resonance, spectrum_lines)
B = bloch.construct_B_field(rf_am, rf_fm=rf_fm, off_resonance=dfz, B1_sensitivity=B1_inhomogeneity)


In [ ]:
B.size * 4 / 1e9 # GB

In [ ]:
B.shape

In [ ]:
type(B)

In [ ]:
k = rf.adiabaticity(rf_pulse_am, rf_pulse_fm, B0, dt)

In [ ]:
k.min()

In [ ]:
# plot RF with bokeh
plot = bkp.figure(width=800, height=400, title='RF pulse')

plot.line(rf_time * 1e3, rf_pulse_am * 1e6, line_width=2)
plot.yaxis.axis_label = 'RF Amplitude (µT)'
plot.y_range = bkm.Range1d(start=0, end=rf_amplitude * 1.1e6)
plot.extra_y_ranges['freq'] = bkm.Range1d(start=-rf_bandwidth * 0.6, end=rf_bandwidth * 0.6)
naxis = bkm.LinearAxis(y_range_name='freq', axis_label='Frequency (Hz)')
plot.add_layout(naxis, 'right')
plot.line(rf_time * 1e3, rf_pulse_fm, line_width=2, y_range_name='freq', color='green')

plot.extra_y_ranges['k'] = bkm.Range1d(start=25, end=36)
kaxis = bkm.LinearAxis(y_range_name='k', axis_label='log(adiabaticity)')
plot.add_layout(kaxis, 'right')
plot.line(rf_time * 1e3, np.log(k), line_width=2, y_range_name='k', color='pink')

plot.xaxis.axis_label = 'Time (ms)'
bkp.show(plot)

In [ ]:
# plot magnetization with bokeh
plot = bkp.figure(width=800, height=400, title='Beff')
plot.line(time, asnumpy(B[:, 0, -1, 0] * 1e6), line_width=2, legend_label='Bx', alpha=0.5)
plot.line(time, asnumpy(B[:, 0, -1, 1] * 1e6), line_width=2, legend_label='By', color='orange', alpha=0.5)
plot.line(time, asnumpy(B[:, 0, -1, 2] * 1e6), line_width=2, legend_label='Bz', color='green')
plot.xaxis.axis_label = 'Time (s)'
plot.yaxis.axis_label = 'B (µT)'
plot.x_range = bkm.DataRange1d(start=0, end=duration)
plot.legend.click_policy = 'hide'
bkp.show(plot)

In [ ]:
mags = bloch.sim(B, T1, T2, duration, dt)

In [ ]:
# plot magnetization with bokeh
plot = bkp.figure(width=800, height=400, title='Magnetization')
plot.line(time, asnumpy(mags[:, 0, -1, 0]), line_width=2, legend_label='Mx', alpha=0.5)
plot.line(time, asnumpy(mags[:, 0, -1, 1]), line_width=2, legend_label='My', color='orange', alpha=0.5)
plot.line(time, asnumpy(mags[:, 0, -1, 2]), line_width=2, legend_label='Mz', color='green')
plot.xaxis.axis_label = 'Time (s)'
plot.yaxis.axis_label = 'Magnetization (ref M0)'
plot.x_range = bkm.DataRange1d(start=0, end=duration)
plot.legend.click_policy = 'hide'
bkp.show(plot)

In [ ]:
# plot magnetization off-resonances with bokeh
title = 'Longitudinal Magnetization with Off-Resonance Pulse'
plot = bkp.figure(width=1000, height=500, title=title)
for offres in range(0, end := mags.shape[1], end // 10):
    alpha = 1 - offres / end
    plot.line(time, asnumpy(mags[:, offres, -1, 2]), line_width=2, legend_label=f'{dfz[offres]:g} Hz',
              alpha=alpha, color='green')
plot.xaxis.axis_label = 'Time (s)'
plot.yaxis.axis_label = 'Magnetization (a.u.)'
plot.x_range = bkm.DataRange1d(start=0, end=duration)
plot.legend.click_policy = 'hide'
bkp.show(plot)

In [ ]:
plot = bkp.figure(width=800, height=400, title=f'Flipped Magnetization Spectrum for {rf_bandwidth} Hz bandwidth')
plot.line(asnumpy(dfz), asnumpy(mags.min(axis=0)[:, -1, -1]), line_width=2)
plot.xaxis.axis_label = 'Off-Resonance Frequency (Hz)'
plot.yaxis.axis_label = 'Magnetization (ref M0)'
plot.y_range = bkm.DataRange1d(start=-1, end=1)
bkp.show(plot)

In [ ]:
freq = asnumpy(dfz)
b1 = asnumpy(B1_inhomogeneity)
minmag = asnumpy(mags.min(axis=0)[..., -1].T)

title = 'Inverted Magnetization Spectrum'
plot = bkp.figure(width=1000, height=500, title=title)
color_mapper = bkm.LinearColorMapper(palette='Viridis256', low=-1, high=1)
image = plot.image([minmag], y=[b1.min()], x=[freq.min()],
                   dh=[b1.max() - b1.min()],
                   dw=[freq.max() - freq.min()], color_mapper=color_mapper)
plot.xaxis.axis_label = 'Off-Resonance Frequency (Hz)'
plot.yaxis.axis_label = 'B1 Inhomogeneity'
plot.x_range = bkm.DataRange1d(start=freq.min(), end=freq.max())
plot.y_range = bkm.DataRange1d(start=b1.min(), end=b1.max())

# add colourbar
color_bar = bkm.ColorBar(color_mapper=color_mapper, location=(0, 0))
color_bar.title = 'Min Magnetization (ref M0)'
plot.add_layout(color_bar, 'right')

# bkp.output_file(f'{title}.html')
# bkp.save(plot)
bkp.show(plot)